Import neccesary libraries for **functionality**

In [ ]:
import tkinter as tk
import ast as ast
import matplotlib.pyplot as plt
import numpy as np

# LibCST basic Functionality Tests
General applications of LibCST (https://github.com/Instagram/LibCST)

In [ ]:
# install libCST
!pip install libcst

In [ ]:
# libCST functionality
import libcst
from libcst.tool import dump

In [ ]:
# test small LibCST conversion
print(dump(libcst.parse_expression("(1 + 2)")))

In [ ]:
# note that only one python file input is assumed, otherwise it only saves the data of the last file
try:
    from google.colab import files
    uploaded = files.upload()
except ImportError as e:
    pass


filenames = uploaded.keys()

for file in filenames:
    data = uploaded[file]

Assorted LibCST tests (requires above file upload to work)

In [ ]:
# convert given file to libCST (https://libcst.readthedocs.io/en/latest/parser.html)
print(dump(libcst.parse_module(data)))

In [ ]:
# or convert and see scope (https://libcst.readthedocs.io/en/latest/scope_tutorial.html)
wrapper = libcst.metadata.MetadataWrapper(libcst.parse_module(data))
scopes = set(wrapper.resolve(libcst.metadata.ScopeProvider).values())
for scope in scopes:
    print(scope)

In [ ]:
# if function name is not foo, print any simple strings found inside (https://libcst.readthedocs.io/en/latest/visitors.html#visit-and-leave-helper-functions)
class FooingAround(libcst.CSTVisitor):
    def visit_FunctionDef(self, node: libcst.FunctionDef) -> bool:
        print(node.name.value)

    def visit_SimpleString(self, node: libcst.SimpleString) -> None:
        print(node.value)


#demo = libcst.parse_module("'abc'\n'123'\ndef foo():\n    'not printed'\ndef blah():\n    'printed'")
demo = libcst.parse_module(data)

_ = demo.visit(FooingAround())

In [ ]:
# print function names
class FuncNames(libcst.CSTVisitor):
    def visit_FunctionDef(self, node: libcst.FunctionDef) -> bool:
        print(node.name.value)


#demo = libcst.parse_module("'abc'\n'123'\ndef foo():\n    'not printed'\ndef blah():\n    'printed'")
demo = libcst.parse_module(data)

_ = demo.visit(FuncNames())

In [ ]:
# edit all function names if encountered (note that changes with transforms happen on leaving a node, so name functions leave_NodeType)

#dict of existing pairs, will be used to replace all calls of old function with calls to new name
# assuming that multiple funcs don't have the same new name
func_name_pairs = dict()

# rename all functions (currently hello[counter of func defs encountered])
class FuncRename(libcst.CSTTransformer):
  count = 0
  def leave_FunctionDef(self, node: libcst.FunctionDef, updated_node: libcst.FunctionDef) -> libcst.FunctionDef:
    self.count += 1
    curr_name = str('hello'+str(self.count))
    func_name_pairs.update({node.name.value: curr_name})
    updated_node = node
    #return updated_node.with_changes(name=libcst.Name(curr_name+libcst.ensure_type(updated_node.name, libcst.Name).value))

    # the name node in function def is a child node, thus to change function name via the FunctionDef parent node, use with_deep_changes via:
    # https://libcst.readthedocs.io/en/latest/nodes.html#libcst.CSTNode.with_deep_changes)
    return updated_node.with_deep_changes(node.name, value=curr_name)

orig = libcst.parse_module(data)
edited = orig.visit(FuncRename())


# print function names
class FuncNames(libcst.CSTVisitor):
    def visit_FunctionDef(self, node: libcst.FunctionDef) -> bool:
        print(node.name.value)

blah = orig.visit(FuncNames())
blah = edited.visit(FuncNames())
print(func_name_pairs)

# 'ast' Python Syntax Analyzer
General applications of "ast" library (https://dev.to/marvinjude/abstract-syntax-trees-and-practical-applications-in-javascript-4a3)

In [ ]:
file_path = "md5 hash.py"

try:
    with open(file_path, "r") as python_file:
        print(f"Opened {file_path} successfully")

        code_string = python_file.read()

        try:
            ast.parse(code_string)
            print(f"Python file: '{file_path}' is syntactically valid")

        except SyntaxError as error:
            print(f"Syntax error in file '{file_path}': {error}")

except FileNotFoundError:
    print(f"Error: The file '{file_path}' was not found.")

except IOError:
    print(f"Error: An error occurred while opening the file '{file_path}'")



# LibCST basic Functionality Tests (Bryan's Branch)
General applications of LibCST (https://github.com/Instagram/LibCST)